<a href="https://colab.research.google.com/github/Martin09/DeepSEM/blob/master/nanowire_yield/2_nw_yield_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2 - Model definition, training and saving
In this notebook we will:
1. Create a new model based on a pre-trained model from the detectron2 repository. 
2. Perform transfer learning to train it on our dataset of labelled SEM images. 
3. Do some quick inference to test our trained model.
4. Save the model to a file for later use.


First, let's check that we are running a GPU instance of Colab:

In [0]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime -> "Change runtime type" menu to enable GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

If you see a prompt above to "Change runtime type" then you are not running a GPU instance. Follow the instructions above to enable the GPU.

# Install detectron2
We will be using Facebook's [detectron2](https://github.com/facebookresearch/detectron2) library to train our model. First we need to install it and its dependencies.

In [0]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version


In [0]:
# install detectron2:
!pip install detectron2==0.1.2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html

In [0]:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, cv2, random, json
from glob import glob
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# Train on a custom dataset

In this section, we will train an existing detectron2 model on our labelled SEM image dataset.

We use [a custom labelled nanowire detection dataset](https://github.com/Martin09/DeepSEM/tree/master/Datasets/NWs-8kMag-subdivided.zip)
which has three classes: 
*   Nanowire
*   Parasitic Growth
*   Nanospade


We'll train a custom object detection model from an existing model pre-trained on the COCO dataset, available in detectron2's model zoo.

Note that the COCO dataset does not have any of these categories so we will have to perform transfer learning to get the model to detect them.

## Prepare our dataset

In [0]:
# Clone the DeepSEM repository
!rm -rf DeepSEM
!git clone https://github.com/Martin09/DeepSEM

# OR

# # Load from my GoogleDrive
# from google.colab import drive
# drive.mount('/content/gdrive/')
# %cd "/content/gdrive/My Drive/LMSC/MachineLearning/Colab"

In [0]:
dataset_dir = './DeepSEM/nanowire_yield/datasets/NWs-8kMag-subdivided/'
!wget https://github.com/Martin09/DeepSEM/raw/master/nanowire_yield/datasets/NWs-8kMag-subdivided.zip
!unzip NWs-8kMag-subdivided.zip -d $dataset_dir
!rm NWs-8kMag-subdivided.zip

## Calculate mean pixel intensity of dataset



In [0]:
# Calcualte dataset mean pixel intensity
images_dir = './DeepSEM/nanowire_yield/datasets/NWs-8kMag-subdivided/'
images = glob(images_dir + '*.png')

min_pixel_intensity = np.infty
max_pixel_intensity = -np.infty
mean_pixel_intensity = 0
for img in images:
    im = cv2.imread(img,cv2.IMREAD_GRAYSCALE)
    min_pixel_intensity = min([np.min(im), min_pixel_intensity])
    max_pixel_intensity = max([np.max(im), max_pixel_intensity])
    mean_pixel_intensity += np.mean(im)
mean_pixel_intensity /= len(images)

print('The min pixel intesity is: {:.2f}'.format(min_pixel_intensity))
print('The max pixel intesity is: {:.2f}'.format(max_pixel_intensity))
print('The mean pixel intesity is: {:.2f}'.format(mean_pixel_intensity))

## Test/train Split

Our dataset consists of our subdivided PNG images in the folder:
```
./DeepSEM/nanowire_yield/datasets/NWs-8kMag-subdivided/
```

Along with our bounding box labels in the form of a labelbox JSON export file located at:
```
'./DeepSEM/nanowire_yield/datasets/export-2020-05-26T15_27_19.519Z.json'
```

Next, we will create a helper function to divide the exported JSON file into two smaller files: a training set and a test set.

In [0]:
# Takes a labelbox exported JSON file and randomly splits it into two other JSON files:
# one for testing and one for validation. Assumes test set should be ~10% of all examples.
def test_train_split(json_file, t_out = 'train.json', v_out = 'val.json'):
    
    # Load Labelbox export file
    with open(json_file) as f:
        dat = json.load(f)

     # Remove empty entries   
    dat = [d for d in dat if d["Label"] != {}]  
    
    # Choose some indices at random for the test set, the rest are for training
    i_test = np.random.choice(len(dat), int(len(dat)*0.1))
    i_train = list(set(list(range(len(dat))))-set(i_test))

    # Split the dataset according to the indices
    dat_train = [dat[i] for i in i_train]
    dat_test = [dat[i] for i in i_test]

    # Write the train.json and val.json files
    with open(t_out, 'w') as fout:
        json.dump(dat_train, fout)
    with open(v_out, 'w') as fout:
        json.dump(dat_test, fout)      
    
    print("Exported {} training images and {} test images!".format(len(i_train),len(i_test)))

In [0]:
json_file = './DeepSEM/nanowire_yield/datasets/export-2020-05-26T15_27_19.519Z.json'
train_out = './DeepSEM/nanowire_yield/datasets/NWs-8kMag-subdivided/nw_yield_train.json'
test_out = './DeepSEM/nanowire_yield/datasets/NWs-8kMag-subdivided/nw_yield_test.json'
test_train_split(json_file, train_out, test_out)

Register the datasets to detectron2, following the [detectron2 custom dataset tutorial](https://detectron2.readthedocs.io/tutorials/datasets.html).
Here, the dataset is in its custom format, therefore we write a function to parse it and prepare it into detectron2's standard format. See the tutorial for more details.


In [0]:
# if your dataset is in COCO format, this cell can be replaced by the following three lines:
# from detectron2.data.datasets import register_coco_instances
# register_coco_instances("my_dataset_train", {}, "json_annotation_train.json", "path/to/image/dir")
# register_coco_instances("my_dataset_val", {}, "json_annotation_val.json", "path/to/image/dir")

from detectron2.structures import BoxMode

# Takes in dataset JSON file and returns a list of dicts in detectron2 format for training
def get_nw_dicts_from_labelbox(json_file):
    category_dict = {'nw':0, 'parasitic':1, 'nanospade':2}

    with open(images_dir + json_file) as f:
        img_anns = json.load(f)

    dataset_dicts = []
    for idx,img in enumerate(img_anns):
        filename = images_dir+'/'+img['External ID']
        height, width = cv2.imread(filename).shape[:2]

        record = {}
        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width

        objs = []
        for anno in img['Label']['objects']:
            bb = anno['bbox']

            obj = {
                "bbox": [bb['left'], bb['top'], bb['width'], bb['height']],
                "bbox_mode": BoxMode.XYWH_ABS,
                "category_id": category_dict[anno['value']],
            }
            objs.append(obj)

        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

Now we register the datasets in detectron2:

In [0]:
from detectron2.data import DatasetCatalog, MetadataCatalog
DatasetCatalog.clear()

for d in ["train", "test"]:
    DatasetCatalog.register("nw_yield_" + d, lambda d=d: get_nw_dicts_from_labelbox("nw_yield_" + d + ".json"))
    MetadataCatalog.get("nw_yield_" + d).set(thing_classes=["nw","parasitic","nanospade"])
nanowire_metadata = MetadataCatalog.get("nw_yield_train")

To verify the data loading is correct, let's visualize the annotations of randomly selected samples in the training set:



In [0]:
dataset_dicts = get_nw_dicts_from_labelbox("nw_yield_train.json")
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])

    visualizer = Visualizer(img[:, :, ::-1], metadata=nanowire_metadata, scale=2.0)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

## Train!

Now, let's transfer learn a coco-pretrained R50-FPN Mask R-CNN model on our SEM dataset. It takes ~35 minutes to train 1000 iterations on Colab Pro's P100 GPUs and a bit longer on Colab's free K80 GPUs.


In [0]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import datetime

timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
output_dir = './DeepSEM/nanowire_yield/output/' + timestamp

cfg = get_cfg()
cfg.OUTPUT_DIR = output_dir
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("nw_yield_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 16
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 16
cfg.SOLVER.BASE_LR = 0.01  # learning rate
cfg.SOLVER.MAX_ITER = 100  # 1000 is decent for initial testing, can do 10k-20k for the final model
cfg.SOLVER.CHECKPOINT_PERIOD = 10000  # Save a checkpoint after every this number of iterations

# Learning rate warmup and decay
cfg.SOLVER.WARMUP_FACTOR = 1/1000.  # Learning starts at BASE_LR * WU_FACTOR
cfg.SOLVER.WARMUP_ITERS = 1000  # Number of iterations for warm-up phase
cfg.SOLVER.WARMUP_METHOD = "linear"
cfg.SOLVER.GAMMA = 0.5
# cfg.SOLVER.STEPS = (100,200,300,)  # List of iteration numbers at which to decrease learning rate by factor GAMMA.
cfg.SOLVER.STEPS = tuple(range(0,cfg.SOLVER.MAX_ITER,1000))

# Validation set
cfg.TEST.EVAL_PERIOD = 0
cfg.TEST.DETECTIONS_PER_IMAGE = 1000

# Don't scale the input images
cfg.INPUT.MIN_SIZE_TRAIN = (0,)  # Keep these data types or might run into issues during inference when loading config file
cfg.INPUT.MAX_SIZE_TRAIN = 99999  # Keep these data types or might run into issues during inference when loading config file

cfg.MODEL.RPN.IN_FEATURES = ['p2', 'p2', 'p3', 'p4', 'p5', 'p6']
cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.25, 0.5, 1.0, 2.0, 4.0, 8.0]]
cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[4], [8], [16], [32], [64], [128]]
cfg.MODEL.PIXEL_MEAN = [58.33, 58.33, 58.33] 
cfg.MODEL.PIXEL_STD = [1.0, 1.0, 1.0]
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 4096
cfg.MODEL.ROI_HEADS.POSITIVE_FRACTION = 0.5
cfg.MODEL.ROI_HEADS.IOU_THRESHOLDS = [0.5] # Intersection over union threshold
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 # We have three classification classes 
cfg.MODEL.RPN.POST_NMS_TOPK_TRAIN = 4000
cfg.MODEL.RPN.PRE_NMS_TOPK_TRAIN = 4000

In [0]:
# Look at training curves in tensorboard:
%load_ext tensorboard
# %reload_ext tensorboard
%tensorboard --logdir ./DeepSEM/nanowire_yield/output

In [0]:
# Start training
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

Hopefully, during training you should start to see the "total_loss" decreasing over time as the model learns.

## Inference & evaluation using the trained model
Now, let's run inference with the trained model on test dataset. First, let's create a predictor using the model we just trained:



In [0]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # Set the testing threshold for this model
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.2     # Non-max supression threshold
cfg.DATASETS.TEST = ("nw_yield_test", )
cfg.TEST.DETECTIONS_PER_IMAGE = 200
cfg.INPUT.MIN_SIZE_TEST = 0
cfg.INPUT.MAX_SIZE_TEST = 99999
predictor = DefaultPredictor(cfg)

Then, we randomly select several samples to visualize the prediction results.

In [0]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_nw_dicts_from_labelbox("nw_yield_test.json")
for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=nanowire_metadata, 
                   scale=2.0, 
                  #  instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])

In [0]:
# # Check the outputs of the neural network manually:
# outputs["instances"].pred_boxes
# outputs["instances"].scores
# outputs["instances"].pred_classes

Now let's save our final model for safe keeping. We will use this model in the next notebook for inference on a full-scale SEM image.

In [0]:
weights_source = cfg.OUTPUT_DIR + '/model_final.pth'
model_name = 'nw_yield_it'+str(trainer.iter)+'_loss0.93.yaml'
model_dest = './DeepSEM/nanowire_yield/trained_models/' + model_name

# Move weights file to "trained_models" folder and update the config file accordingly
weights_dest = model_dest[:-5] + '.pth'
!cp $weights_source $weights_dest
cfg.MODEL.WEIGHTS = weights_dest

# Save the config file alongside the weights file
confi_dest = model_dest
with open(confi_dest, "w") as text_file:
    text_file.write(cfg.dump())

We can also evaluate its performance using AP metric implemented in COCO API.

In [0]:
# ## Not working yet, probably don't have dataset in proper COCO format
# from detectron2.evaluation import COCOEvaluator, inference_on_dataset
# from detectron2.data import build_detection_test_loader
# evaluator = COCOEvaluator("nw_yield_train", cfg, False)#, output_dir="./output/")
# val_loader = build_detection_test_loader(cfg, "nw_yield_train")
# inference_on_dataset(trainer.model, val_loader, evaluator)
# # another equivalent way is to use trainer.test